In [36]:
from flask       import Flask, request, jsonify
from Levenshtein import ratio, jaro, distance
from collections import Counter

import pandas as pd
import requests
import subprocess
import gzip
import json
import math
import os
import re

def set_default_path(directory: str | None = 'Event-Aggregator') -> str:
    """Changes the working directory to """
    while not os.getcwd().endswith(directory):
        os.chdir('..')
        if len(os.getcwd()) == 0:
            raise FileExistsError
    return os.getcwd()

default_path = os.path.join(set_default_path() ,'Production_Environment')
server_path  = os.path.join(default_path,'Servers') 
collectors_path = os.path.join(server_path,'Collectors')
events_path  = os.path.join(collectors_path,'Events')
data_path    = os.path.join(events_path,'Data')
File_path    = os.path.join(data_path, 'Final_Events.csv')

In [ ]:
def query(url, data, mode):
    match mode:
        case 'POST':
            item = requests.post(url,json=data)
            item = json.loads(item.content.decode())
        case 'GET':
            item = requests.get(url)
            item = json.loads(item.content.decode())
    return item

In [ ]:
WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [91]:
df = pd.read_csv(File_path)

In [92]:
# metric = df['Title']\
#         .apply(func = lambda s: s.replace(' ','')) \
#         .apply(func = lambda s: s.lower()) \
#         .sort_values(ascending=False)\
#         [:10].index

In [90]:
# Makes requests to the front server
url  = "http://127.0.0.1:5000/search"
data = {"QUERY"  : 'Esports',
        "BY"     : "Title",
        "NUMBER" : 5
        }

item = query(url,data, mode='POST')
item

{'15': {'Address1': 'Paradise Valley Community Center, 17402 N 40th St',
  'Address2': 'Phoenix, AZ',
  'Desc': 'Join us every Wednesday from 5p - 7p and Saturdays 11a-2p for Esports Weekly Open Play! Compete against your friends or hone your skills to become the best player in Phoenix! Open Game Room...',
  'Main Page': 'https://www.jackcentral.org/local-events/?_evDiscoveryPath=/event/37571249a-esports-weekly-open-play-pvcc',
  'Start Date': 'Mar 2',
  'Title': 'Esports - Weekly Open Play @ PVCC',
  'When': 'Feb 26, 5 PM – Mar 26, 7 PM MST',
  'image': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSl14ALJrp2Ho2cr3yiRdemxrv0zUJ4vRDmhZM-KguJnw&s=10',
  'thumb': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRdu84pnoqrDA8eRYotRmVBtUt4KLrORUHWbm3AvkfGM_xprSrVPR_uvsY&s'},
 '16': {'Address1': 'Hartford Public Library, Albany Branch, 1250 Albany Ave',
  'Address2': 'Hartford, CT',
  'Desc': 'Experience the excitement of competitive gaming! Prepare to engage in competition, 